## `array.array` and `np.array`

### array.array

The list type is nice and very flexible, but if you need to store many many (millions) of floating point variables, array.array is a better option. It stores just the bytes representing the type, so its just like a contiguous C array of things in RAM, and also just like a numpy array. 

`array.array` IS mutable, and you dont need to allocate ahead of time (reallocation will be done).

The constructor is: 

`array(typecode [, initializer]) -- create a new array`



In [23]:
from array import array
from random import random
#generator expression instead of list comprehension
floats_aa=array('d', (random() for i in range(10**8)))

In [24]:
floats_aa.itemsize

8

In [25]:
type(floats_aa), floats_aa[5]

(array.array, 0.16155087887707942)

### Numpy arrays

I'll refer you to their quickstart in case you have not already worked with them. Bottom line: they need a size!

https://docs.scipy.org/doc/numpy-dev/user/quickstart.html


In [26]:
import numpy as np
af_array = np.array([1,2,3], dtype="float64")

In [27]:
af_array

array([ 1.,  2.,  3.])

In [28]:
af_array.dtype, af_array.shape, af_array.ndim

(dtype('float64'), (3,), 1)

In [29]:
af_array_2 = np.ones(shape=10, dtype=int)

In [30]:
af_array_2

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [31]:
af_array_3 = np.ones(shape=(10,), dtype="float32")

In [32]:
af_array_3

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.], dtype=float32)

## Where do trees arise

A tree is:

- a hierarchical data structure that has a bunch of items,
- each of which may have a value
- some of which may point to other such items, and some that dont (leaf nodes)
- each item is pointed to by exactly one other item, with the sole exception of the root.


Trees arise everywhere:

- in parsing of code
- evolutionary trees in biology
- language origin trees
- unix file system
- html tags on this page

Just like with lists, one can consider looking at a tree in two ways: a collection of nodes, or a tree with a root and multiple sub-tree's.


### Parsing into a Tree

Consider writing a small calculator language, which takes calculator expressions written in an prefix format, and calculates them for us.

Parsing is the idea of generating an expression tree from code text. A parser has 2 components:

1. a lexical analyser or lexer which converts code into tokens
2. a syntactic analyzer which constructs an expression tree from these tokens

Here is code for our infix calculator. We have no variables, and no precedence, to keep things simple. Brackets need to be explicit.

`(+ 1 (* 2 3)) = 7`

 We will need to **parse** this expression: the output will be a parse tree. For simplicity we will assume that all operations are binary, and thus represent the parse tree is a binary tree...

In [33]:
from IPython.display import Image

In [44]:
import uuid
class BinaryTree:
    def __init__(self, value, parent=None):
        self.value = value
        self.parent = parent
        self.uuid= uuid.uuid4()
        self.left = None
        self.right = None

    def addLeftChild(self, value): 
        n = BinaryTree(value, self)
        self.left = n
        return n
        
    def addRightChild(self, value):
        n = BinaryTree(value, self)
        self.right = n
        return n
        
    def hasLeftChild(self):
        return self.left != None

    def hasRightChild(self):
        return self.right != None

    def isLeftChild(self):
        return self.parent and self.parent.left == self

    def isRightChild(self):
        return self.parent and self.parent.right == self

    def isRoot(self):
        return not self.parent

    def isLeaf(self):
        return not (self.right or self.left)
    
    def preorder(self):
        if self.isLeftChild():
            yield (self.parent.value, self.parent.uuid, self.value, self.uuid, "left")
        elif self.isRightChild():
            yield (self.parent.value, self.parent.uuid, self.value, self.uuid, "right")
        if self.hasLeftChild():
            for v in self.left.preorder():
                yield v
        if self.hasRightChild():
            for v in self.right.preorder():
                yield v


In [45]:
junk = BinaryTree(5)
l = junk.addLeftChild(3)
r = junk.addRightChild(31)
l.addLeftChild(8)
l.addRightChild(3)
r.addRightChild(22)

In [46]:
junk.hasLeftChild(), junk.left, junk.left.value

(True, <__main__.BinaryTree at 0x13a3ae908>, 3)

In [47]:
l.value, l.right.value

(3, 3)

In [48]:
junk.preorder()

<generator object BinaryTree.preorder at 0x13a38c468>

In [49]:
import subprocess
def plotbtree(btree, fname):
    start="""
    digraph G {
    nodesep=0.3;
    ranksep=0.2;
    margin=0.1;
    node [shape=circle];
    edge [arrowsize=0.8];
    """
    end = "}"
    assert btree.isRoot(), "Must start at root"
    lines=""
    labeldict={}
    for t in btree.preorder():
        if not t[1] in labeldict:
            labeldict[t[1]]=t[0]
        if not t[3] in labeldict:
            labeldict[t[3]]=t[2]
        lines = lines + "\"{}\" -> \"{}\"[side={}];\n".format(t[1],t[3],t[4])
    end="\n".join(["\"{}\"[label=\"{}\"]".format(k,v) for (k, v) in labeldict.items()])+end
    start = start + lines +end
    with open(fname+".dot","w") as fd:
        fd.write(start)
    #comment out if needed
    subprocess.call(['./treeplot.sh', fname])

You can plot resultant graphs online as well: http://sandbox.kidstrythisathome.com/erdos/

In [53]:
plotbtree(junk, "junk")

FileNotFoundError: [Errno 2] No such file or directory: './treeplot.sh'

In [51]:
!cat junk.dot


    digraph G {
    nodesep=0.3;
    ranksep=0.2;
    margin=0.1;
    node [shape=circle];
    edge [arrowsize=0.8];
    "5f4f1856-4302-4ada-9249-c1eccdcdfae7" -> "95c49dd0-0fe1-46c7-963d-c525df0433e3"[side=left];
"95c49dd0-0fe1-46c7-963d-c525df0433e3" -> "941d5d69-f47d-4d7f-9eee-613cf916ca12"[side=left];
"95c49dd0-0fe1-46c7-963d-c525df0433e3" -> "34f93d4c-7b5a-4896-9acb-7a197731d405"[side=right];
"5f4f1856-4302-4ada-9249-c1eccdcdfae7" -> "78efd193-a915-4abd-b087-4e7370dea759"[side=right];
"78efd193-a915-4abd-b087-4e7370dea759" -> "c58e74c7-fe06-47d1-9cf0-cd634a91b2d7"[side=right];
"941d5d69-f47d-4d7f-9eee-613cf916ca12"[label="8"]
"78efd193-a915-4abd-b087-4e7370dea759"[label="31"]
"95c49dd0-0fe1-46c7-963d-c525df0433e3"[label="3"]
"34f93d4c-7b5a-4896-9acb-7a197731d405"[label="3"]
"5f4f1856-4302-4ada-9249-c1eccdcdfae7"[label="5"]
"c58e74c7-fe06-47d1-9cf0-cd634a91b2d7"[label="22"]}

In [43]:
Image("junk.png")


In [ ]:
!cat treeplot.sh

In [ ]:
!cat tree.gv

### Lexing

In [ ]:
import string
#some code from composing programs
_WHITESPACE = set(' \t\n\r')
_NUMERAL_STARTS = set(string.digits) | set('+-.')
_SINGLE_CHAR_TOKENS = set("()")
_TOKEN_END = _WHITESPACE | _SINGLE_CHAR_TOKENS
_SYMBOL_STARTS = set('%*/+-^') 
_SYMBOL_INNERS = set(string.digits) | set('+-.')

def next_token(expression, pos):
    while pos < len(expression):
        c = expression[pos]
        if c in _WHITESPACE:
            pos += 1
        elif c in _SINGLE_CHAR_TOKENS:
            #print(">c",c)
            return c, (pos + 1)
        else:#a numerical expression or operator
            j = pos
            while j < len(expression) and expression[j] not in _TOKEN_END:
                j += 1
            #print('c>',expression[pos:j])
            return expression[pos:j], min(j, len(expression))
    return None, len(expression)

def valid_symbol(s):
    if len(s) == 0 or s not in _SYMBOL_STARTS:
        return False
    return True

def lex(expression):
    """
    An expression parser
    
    >>> lex('(+ 1 (* 23 3))')
    ['(', '+', 1, '(', '*', 23, 3, ')', ')']
    >>> lex('(+ 1 (* 23 -3))')
    ['(', '+', 1, '(', '*', 23, -3, ')', ')']
    >>> lex('(+ 1 (* 23 -3e03))')
    ['(', '+', 1, '(', '*', 23, -3000.0, ')', ')']
    >>> lex('(+ (* 2 3) (* 23 3))')
    ['(', '+', '(', '*', 2, 3, ')', '(', '*', 23, 3, ')', ')']
    """
    elen = len(expression)
    tokens=[]
    text, i = next_token(expression, 0)
    while text is not None:
        #print(text, expression)
        if text in _SINGLE_CHAR_TOKENS:
            tokens.append(text)
        elif text[0] in _SYMBOL_STARTS and valid_symbol(text):
            tokens.append(text)
        elif text[0] in _NUMERAL_STARTS:
            try:
                tokens.append(int(text))
            except ValueError:
                try:
                    tokens.append(float(text))
                except ValueError:
                    raise ValueError("invalid numeral: {0}".format(text))
        
        text, i = next_token(expression, i)
    return tokens

In [ ]:
lex('(+ (* 2 3) (* 23 3))')

In [ ]:
from doctest import run_docstring_examples as dtest
dtest(lex, globals(), verbose=True)

### Syntactic Analysis 

...to get the tree..

In [ ]:
def syn(tokens):
    currentTree = None
    stack = []
    for i, c in enumerate(tokens):
        if c == '(':
            if currentTree is not None:
                if currentTree.left is None:
                    ntree = currentTree.addLeftChild(tokens[i+1])
                else:
                    ntree = currentTree.addRightChild(tokens[i+1])
            else:
                ntree = BinaryTree(tokens[i+1])
            stack.append(ntree)
            currentTree = ntree
        elif c in _SYMBOL_STARTS:
            assert currentTree.value == c, "op should follow ("
        elif c not in _SYMBOL_STARTS | _SINGLE_CHAR_TOKENS: #number
            if currentTree.left is None:
                leaf = currentTree.addLeftChild(c)
            else:
                leaf = currentTree.addRightChild(c)
        elif c==')':
            if len(stack) > 1:
                stack.pop()
            currentTree=stack[-1]
    stack.pop()
    return currentTree

In [ ]:
t = syn(lex('(+ (* 2 3) (* 23 3))'))

In [ ]:
t.value, t.left.value, t.right.value

In [ ]:
syn(lex('(+ 1 (* 23 3))'))

In [ ]:
syn(lex('(+ (* 2 3) (* 23 3))'))

In [ ]:
plotbtree(syn(lex('(+ (* 2 3) (* 23 3))')), 'exptree')

In [ ]:
!cat exptree.dot

In [ ]:
Image('exptree.png')

### Tree traversal

We need to decide how we plan to tracerse the tree. There are 3 choices, based on **an implicit ordering** of the tree from left to right

- In-order: visit left-subtree, then current node, then right sub tree
- Post-order: do child nodes first, and then current node
- Pre-order: do current node, then children

By visit ot do i mean the execution of some operation. For example, consider the filesystem tree. Order does not matter there, but say removing files in a directory before remoc=ving a directory makes sense: this is post-order. similarly to find size of a folder.

Pre-order is rarely used (although we did use it to write a pre-order iterator for the binary tree just now to output stuff in the dot language).

In [ ]:
def preorder_traverse(tree, func):
    if tree:
        func(tree.value)
        preorder_traverse(tree.left, func)
        preorder_traverse(tree.right, func)
def inorder_traverse(tree, func):
    if tree:
        inorder_traverse(tree.left, func)
        func(tree.value)
        inorder_traverse(tree.right, func)
def postorder_traverse(tree, func):
    if tree:
        r1=postorder_traverse(tree.left, func)
        r2=postorder_traverse(tree.right, func)
        return func(tree.value, r1, r2)

Ok, so lets support some simple ops...by delegating our "languages" ops to python :-)

In [ ]:
from operator import add, sub, mul, pow, mod, truediv
OPDICT={
    '+': add,
    '-': sub,
    '*': mul,
    '/': truediv,
    '%': mod,
    '^': pow
}
def eval_exp(v, x, y):
    if x is not None and y is not None:
        return OPDICT[v](x,y)
    else:
        return v

In [ ]:
postorder_traverse(syn(lex('(+ 1 (* 23 3))')), eval_exp)

In [ ]:
postorder_traverse(syn(lex('(+ 1 (* 23 (/ 1 .3)))')), eval_exp)

In [ ]:
postorder_traverse(syn(lex('(+ (* 2 3) (* 23 3))')), eval_exp)